In [ ]:
import json
import numpy as np
from IPython.display import display, HTML
import pandas as pd
pd.set_option("display.precision", 3)

In [ ]:
file_path = "../result.json"

result_data = []

# Open the file and load JSON data
with open(file_path, 'r') as json_file:
    for line in json_file:
        try:
            result_data.append(json.loads(line))
        except:
            continue

In [ ]:
planner_names = list(set([d['planner_name'] for d in result_data]))

planner_names = [
    "MTGTaskPlanner",
    "MTGTaskPlannerWithGMM",
    # "DynamicMTGTaskPlannerWithGMM_25.0",
    # "DynamicMTGTaskPlannerWithGMM_50.0",
    # "DynamicMTGTaskPlannerWithGMM_75.0",
    # "DynamicMTGTaskPlannerWithGMM_100.0",
    # "DynamicMTGTaskPlannerWithGMM_125.0",
#     "DynamicMTGTaskPlannerWithGMM_150.0",
    "MTGTaskPlannerWithAtlas",
    # "DynamicMTGTaskPlannerWithAtlas_25.0",
    # "DynamicMTGTaskPlannerWithAtlas_50.0",
    # "DynamicMTGTaskPlannerWithAtlas_75.0",
    # "DynamicMTGTaskPlannerWithAtlas_100.0",
    # "DynamicMTGTaskPlannerWithAtlas_125.0",
#     "DynamicMTGTaskPlannerWithAtlas_150.0",
]

metrics = [
    'motion_planning_time',
    'task_planning_time',
    'updating_time',
    'solution_length',
    'num_attempts',
]

planner_stats = {
                    planner_name: {
                        'num' : 0, 
                        'num_success' : [], 
                        'motion_planning_time' : [],
                        'task_planning_time' : [],
                        'updating_time' : [],
                        'solution_length' : [],
                        'num_attempts' : [],
                        'total_planning_time' : [],
                    } 
                 
                 for planner_name in planner_names
                }

for d in result_data:
    if d['planner_name'] in planner_stats:
        dic = planner_stats[d['planner_name']]
        dic['num'] += 1
        dic['num_success'].append(d['success'] == "true")
        if d['success'] == "true":
            dic['motion_planning_time'].append(d['motion_planning_time'])
            dic['task_planning_time'].append(d['task_planning_time'])
            dic['total_planning_time'].append(d['total_planning_time'])
            dic['updating_time'].append(d['updating_time'])
            dic['solution_length'].append(d['solution_length'])
            dic['num_attempts'].append(d['num_attempts'])

# print(
#         "{0: >6}".format("Planner Name"), 
#         "{0: >6}".format("Success Rate"),
#         "{0: >6}".format("Mean Task Planning Time"),
#         "{0: >6}".format("Mean Motion Planning Time"),
#         "{0: >6}".format("Mean Total Planning Time"),
#         "{0: >6}".format("Mean Update time"),
#         "{0: >6}".format("Mean solution length"),
#         "{0: >6}".format("Mean number of attempts"),
#     )        

output_dict = []
for planner_name in planner_names:
    stats = planner_stats[planner_name]

    # print(
    #         "{0: >37}".format(planner_name), 
    #         "{:.3f}%".format(np.mean(stats['num_success']) * 100.0) , 
    #         "{:.3f}".format(np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['total_planning_time']) / (np.array(stats['num_attempts'])) )),
    #         "{:.3f}".format(np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )), 
    #         "{:.3f}".format(np.mean((np.array(stats['num_attempts'])) )),

    #     )
    current_planner_eval_dict = {}
    current_planner_eval_dict["Planner Name"] = planner_name
    current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
    current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
    current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )
    current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
    output_dict.append(current_planner_eval_dict)
df = pd.DataFrame(output_dict).set_index('Planner Name')

display(df)

In [ ]:
import pandas as pd

df = pd.DataFrame(result_data)
df

In [ ]:
for groupname, group in df.groupby('planner_name', sort=False):
    print(groupname)
    display(group.describe().loc[['mean', 'std', '50%', 'max']])